In [1]:
import json
import pandas as pd
# Opening JSON file
f = open('/home/ferdinand_t/data/CMS_2010_to_June_2022_ENGLISH.json')
# returns JSON object as
# a dictionary
data = json.load(f)
df = pd.DataFrame.from_dict(data)

In [2]:
df['firstKeyword'] = df['keywords'].apply(lambda x: x[0]['name'] if len(x) != 0 else None)
# #df['secondKeyword'] = df['keywords'].apply(lambda x: x[1]['name'] if len(x) > 1 else None)
# #df['thirdKeyword'] = df['keywords'].apply(lambda x: x[2]['name'] if len(x) > 2 else None)
# #df['fourthKeyword'] = df['keywords'].apply(lambda x: x[3]['name'] if len(x) > 3 else None)

df['cleanFocusCategory'] = df['thematicFocusCategory'].apply(lambda x: x['name'] if x is not None else x)

# #df = df[['firstKeyword', 'secondKeyword', 'thirdKeyword', 'fourthKeyword', 'thematicFocusCategory', 'cleanFocusCategory']]

df = df[['firstKeyword', 'keywordStrings', 'cleanFocusCategory']]

df_clean = df.copy()
df_clean = df_clean.dropna()
# df_clean['cleanKeywordStrings'] = [' '.join(map(str, l)) for l in df_clean['keywordStrings']]

In [3]:
import gensim
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("/home/ferdinand_t/Downloads/GoogleNews-vectors-negative300.bin", binary=True)

In [4]:
# Count vectorization of text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
corpus_train, corpus_test, label_train, label_test, indices_train, indices_test = train_test_split(df_clean['keywordStrings'], df_clean['cleanFocusCategory'].astype(str), df_clean.index, test_size=0.33, random_state=0) 
# Ticket Data
corpus = corpus_train.astype(str)
 
# Creating the vectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1),stop_words='english')
 
# Converting the text to numeric data
X = vectorizer.fit_transform(corpus)
 
#print(vectorizer.get_feature_names())
# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
CountVectorizedData['cleanFocusCategory']=label_train.values
print(CountVectorizedData.shape)
CountVectorizedData.head()

(53541, 31881)


/home/ferdinand_t/venv/lib64/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000,007,01,03,04,05,08,0rg,10,100,...,özil,özlem,özoguz,øystein,út,überall,ünal,ünker,żurek,cleanFocusCategory
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Conflicts
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Conflicts
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Politics
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Politics
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Conflicts


In [5]:
WordsVocab=CountVectorizedData.columns[:-1]

In [12]:
import numpy as np
X_corpus_train = vectorizer.transform(corpus_train.astype(str))
CountVecData=pd.DataFrame(X_corpus_train.toarray(), columns=vectorizer.get_feature_names())
    
    # Creating empty dataframe to hold sentences
W2Vec_Data_temp=pd.DataFrame()
    
Sentence_1=[[wv[word] if word in wv.key_to_index.keys() else np.zeros(300) for word in WordsVocab[CountVecData.iloc[i , :]>=1]] for i in range(CountVecData.shape[0])]
W2Vec_Data_temp=W2Vec_Data_temp.append(pd.DataFrame(Sentence_1))
Sentence_2=[[wv[word.capitalize()] if word.capitalize() in wv.key_to_index.keys() and word not in wv.key_to_index.keys()  else np.zeros(300) for word in WordsVocab[CountVecData.iloc[i , :]>=1]] for i in range(CountVecData.shape[0])]
W2Vec_Data_temp=W2Vec_Data_temp.append(pd.DataFrame(Sentence_2))
test_sum_df = W2Vec_Data_temp.groupby(W2Vec_Data_temp.index).sum()
#W2Vec_Data = test_sum_df.sum(axis=1)
test_sum_df[test_sum_df.applymap(lambda x: np.allclose(x, 0))] = np.nan
W2Vec_Data = test_sum_df.apply(lambda x: np.mean(x[x.notnull()]), axis=1)
W2Vec_Data_df = pd.DataFrame(W2Vec_Data)
W2Vec_Data_df_final = W2Vec_Data_df[0].apply(pd.Series)
W2Vec_Data_corpus_train = W2Vec_Data_df_final

/home/ferdinand_t/venv/lib64/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
#TODO THEN I DONT HAVE TO CREATE THE DATAFRAME BEFORE THAT STEP
# Adding the target variable
W2Vec_Data_corpus_train.reset_index(inplace=True, drop=True)
W2Vec_Data_corpus_train['cleanFocusCategory']=label_train.values
 
# Assigning to DataForML variable
DataForML_corpus_train=W2Vec_Data_corpus_train
DataForML_corpus_train.loc[DataForML_corpus_train.isna().any(axis=1),0:299] = 0
DataForML_corpus_train.shape

(53541, 301)

In [14]:
# Separate Target Variable and Predictor Variables
TargetVariable_corpus_train=DataForML_corpus_train.columns[-1]
Predictors_corpus_train=DataForML_corpus_train.columns[:-1]
 
X_corpus_train=DataForML_corpus_train[Predictors_corpus_train].values
y_corpus_train=DataForML_corpus_train[TargetVariable_corpus_train].values

In [16]:
import numpy as np
X = vectorizer.transform(corpus_test.astype(str))
CountVecData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    
    # Creating empty dataframe to hold sentences
W2Vec_Data_temp=pd.DataFrame()
    
Sentence_1=[[wv[word] if word in wv.key_to_index.keys() else np.zeros(300) for word in WordsVocab[CountVecData.iloc[i , :]>=1]] for i in range(CountVecData.shape[0])]
W2Vec_Data_temp=W2Vec_Data_temp.append(pd.DataFrame(Sentence_1))
Sentence_2=[[wv[word.capitalize()] if word.capitalize() in wv.key_to_index.keys() and word not in wv.key_to_index.keys()  else np.zeros(300) for word in WordsVocab[CountVecData.iloc[i , :]>=1]] for i in range(CountVecData.shape[0])]
W2Vec_Data_temp=W2Vec_Data_temp.append(pd.DataFrame(Sentence_2))
test_sum_df = W2Vec_Data_temp.groupby(W2Vec_Data_temp.index).sum()
#W2Vec_Data = test_sum_df.sum(axis=1)
test_sum_df[test_sum_df.applymap(lambda x: np.allclose(x, 0))] = np.nan
W2Vec_Data = test_sum_df.apply(lambda x: np.mean(x[x.notnull()]), axis=1)
W2Vec_Data_df = pd.DataFrame(W2Vec_Data)
W2Vec_Data_df_final = W2Vec_Data_df[0].apply(pd.Series)
W2Vec_Data = W2Vec_Data_df_final

In [17]:
#TODO THEN I DONT HAVE TO CREATE THE DATAFRAME BEFORE THAT STEP
# Adding the target variable
W2Vec_Data.reset_index(inplace=True, drop=True)
W2Vec_Data['cleanFocusCategory']=label_test.values
 
# Assigning to DataForML variable
DataForML=W2Vec_Data
DataForML.loc[DataForML.isna().any(axis=1),0:299] = 0
DataForML.shape

(26372, 301)

In [18]:
# Separate Target Variable and Predictor Variables
TargetVariable=DataForML.columns[-1]
Predictors=DataForML.columns[:-1]
 
X=DataForML[Predictors].values
y=DataForML[TargetVariable].values

In [11]:
# # Split the data into training and testing set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

(19779, 300)
(19779,)
(6593, 300)
(6593,)


In [19]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

model = LinearSVC()
model.fit(X_corpus_train, y_corpus_train)
y_pred = model.predict(X)
accuracy_score(y,y_pred)

0.6494387987259215